In [2]:
import pandas as pd

# Kiểm tra dữ liệu đầu vào
df = pd.read_csv('/kaggle/input/btcusdt-5m-full/BTCUSDT_5m_full.csv')
print(f"Data shape: {df.shape}\nColumns: {df.columns.tolist()}\nSample:\n{df.head()}")

Data shape: (775824, 6)
Columns: ['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
Sample:
             timestamp     Open     High      Low    Close    Volume
0  2017-08-17 04:00:00  4261.48  4280.56  4261.48  4261.48  2.189061
1  2017-08-17 04:05:00  4261.48  4261.48  4261.48  4261.48  0.000000
2  2017-08-17 04:10:00  4261.48  4261.48  4261.48  4261.48  0.000000
3  2017-08-17 04:15:00  4261.48  4264.88  4261.48  4261.48  0.484666
4  2017-08-17 04:20:00  4264.88  4266.29  4264.88  4266.29  2.328570


In [3]:
!pip install ta  # Thư viện tính technical indicators


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=6d40cdf135f658d3238d0b898e8a08c1a6209d6f397c5ed54b146356ddc7de00
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [4]:
%%writefile /kaggle/working/RWKV-TS/configs/train_config.yaml

model:
  # === Input/Output ===
  enc_in: 22
  c_out: 1
  seq_len: 288
  pred_len: 12
  
  # === LSTM Architecture ===
  d_model: 64
  e_layers: 4
  dropout: 0.3

  # === Time Features ===
  time_features: 0  

data:
  path: "/kaggle/input/btcusdt-5m-full/BTCUSDT_5m_full.csv"
  freq: "5T"
  train_ratio: 0.8             # 80% training, 20% validation

training:
  epochs: 10                    #30
  batch_size: 128              # Increased batch size for GPU
  lr: 0.001                     # Reduced learning rate
  patience: 15                 # More patience for LSTM
  min_delta: 0.005             # More sensitive early stopping
  log_dir: "logs"
  checkpoint_dir: "checkpoints"
  device: "cuda"               
  clip_grad: 1.0               # Gradient clipping
  weight_decay: 0.0001         # L2 regularization
  resume: "auto"

Overwriting /kaggle/working/RWKV-TS/configs/train_config.yaml


In [5]:
!python /kaggle/working/RWKV-TS/src/train.py \
  --config /kaggle/working/configs/btc_5m_config.yaml \   

2025-05-10 18:54:21.482270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746903261.671792      65 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746903261.728619      65 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
lstm
Epoch 1/10: 100%|█████████| 4843/4843 [02:19<00:00, 34.78batch/s, loss=0.000566]
[Eval] MSE: 0.0008 | MAE: 0.0181 | RMSE: 0.0276 | SMAPE: 7.42% | R2: 0.9982
Epoch 2/10: 100%|█████████| 4843/4843 [02:17<00:00, 35.18batch/s, loss=0.000345]
[Eval] MSE: 0.0005 | MAE: 0.0140 | RMSE: 0.0228 | SMAPE: 5.88% | R2: 0.9988
Epoch 3/10: 100%|█████████| 4843/4843 [02:16<00:00, 35.46batch/s, loss=0.000325]
[Eval] MSE: 0.0005 | MAE: 0.0138 | RM